# Crear aplicacion para clasificar gatos y perros

In [9]:
#Folder Structure
import os

#Config
bs = 64
ep = 50
dim1 = dim2 = 256
dataset_path = os.path.join('./data','raw-img')


print(os.listdir(dataset_path))

from translate import translate 

['pecora', 'cane', 'ragno', 'cavallo', 'gallina', 'scoiattolo', 'farfalla', 'gatto', 'elefante', 'mucca']


In [32]:
!pip3 install keras
!pip3 install tensorflow
!pip3 install pillow

     |████████████████████████████████| 3.1 MB 1.1 MB/s eta 0:00:01


In [17]:
#importing files for CNN
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout



In [16]:
#model init
model = Sequential()

#conv layer
model.add(Convolution2D(
  64, # numero de filtros de salida
  (3, 3), # dimension de matriz de convolucion
  activation='relu', 
  input_shape=(dim1,dim2,3) # definicion de dimesion de entrada
))

#pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

#additional pairs
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#flat layer
model.add(Flatten())

In [18]:
#cnn connection

#hidden layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.2))

#output layer
model.add(Dense(units=10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 127, 127, 64)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 30, 30, 128)     

In [19]:
#brigning all of it together
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
from tensorflow.keras import applications
model_2 = applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    classifier_activation='softmax'
)
model_2.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
from keras.preprocessing.image import ImageDataGenerator


In [24]:
image_data_generator = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [38]:
train_generator = image_data_generator.flow_from_directory(
    dataset_path,
    target_size = (dim1, dim2),
    batch_size = bs,
    subset="training"
)

validation_generator = image_data_generator.flow_from_directory(
    dataset_path,
    target_size = (dim1, dim2),
    batch_size = bs,
    subset="validation"
)

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [46]:
#Indices
'''
cane  -   Dog
cavallo  -  Horse
elefante  -  Elephant
farfalla  -  Butterfly
gallina  -  Hen
gatto  -  Cat
mucca  -  Cow
pecora  -  Sheep
ragno  -  Spider
scoiattolo  -  Squirrel
'''

train_generator.class_indices

{'cane': 0,
 'cavallo': 1,
 'elefante': 2,
 'farfalla': 3,
 'gallina': 4,
 'gatto': 5,
 'mucca': 6,
 'pecora': 7,
 'ragno': 8,
 'scoiattolo': 9}

In [50]:
history = model.fit(
                    train_generator,
                    steps_per_epoch = (20947 / bs),
                    epochs = ep,
                    validation_data = validation_generator,
                    validation_steps = (5232/bs),
                    workers=5,)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.